# Make a Library Management System which has the following functionalities:

## Database of Books that Contains | Dictionary

● Book Name

● Book ID

● Quantity

● Genre

● Price

● Author

## Database of Students
● Student Name

● Student ID

● Graduation Year

● Graduation Degree

● Books Issued | List of Books ID with Issue and Expiry Date

**Dictionary structure of books.json**

```python

books = {
    'book_id_1': {
        'name'     : # string   value
        'quantity' : # integer  value
        'genre'    : # string   value
        'price'    : # floating value
        'author'   : # string   value
    },
    'book_id_2': { ... },
    ...
}
```

**Dictionary structure of students.json**

```python

students = {
    'student_id_1': {
        'name'            : # string   value
        'graduation year' : # integer  value
        'degree'          : # string   value
        'taken'           : # integer values --> 0, 1, 2
        'issued'          : [ ('book_ids','expire_date'), ... ] # list of book ids may varies
    },
    'student_id_2': { ... },
    ...
}   
```

## Import Libraries

In [1]:
import json as js
import pandas as pd
from IPython.display import clear_output
from datetime import datetime  as dt
from datetime import timedelta as td

## Read and Write Json Files function

In [2]:
# reads json file and return dictionary 
def read_file( name = 'book' ):
    
    # writing extension of file as .json
    name += 's.json'
    
    # Opening json file in read mode
    fr    = open( name ,'r')

    # Loading json file into dictionary
    r     = js.loads( fr.read() )

    # Closing the file
    fr.close()
    
    # Return dictionary
    return r

In [3]:
# write dictionary data into json file 
def write_file(var, name = 'book' ):
    
    # writing extension of file as .json
    name += 's.json'
    
    # Opening json file in write mode
    fw    = open( name ,'w')

    # Dumping dictionary into json file  
    fw.write(js.dumps(var))

    # Closing the file
    fw.close()

## Choice function

In [4]:
# Program for choice return choice or False for invalid input 
def choices(limit):
    
    # Taking input of choice from user
    choice = input('Enter your choice number: ')
    
    # try block to catch error for int function
    try:
        
        # convert str to int if error then go to except block
        choice = int(choice)
        
        # valid choice and return choice as o/p
        if 0 < choice <= limit:
            return choice
        
        # invalid choice and return False as o/p
        else :
            print('Enter valid number try again ....')
            print('-'*40)
            return False
        
    # except block for int errors and return False as o/p
    except:
        print('Invalid input try again ....')
        print('-'*40)
        return False

# Getting input of book_id and scan_id   
def inputs():
    
    # While Loop until inputs are valid
    while True:
        
        #
        try:
            book_id    = input('Scan / Type Book id: ').upper()

            if not book_id in books    :

                print('Book id not found')
                continue


            stud_id    = input('Scan / Type student id: ').upper()

            if not stud_id in students :

                print('Student id not found')
                continue

            break
        except:
            print('Try Again ... ')
            
    return (book_id,stud_id)

# Book Issue and Reissue Application

In [5]:
def book_avail():
    # Loop this message until exit
    while True:
        # printing statement
        print('Check Availiability')
        print('1. Search by Name')
        print('2. Search by Author')
        print('3. Search by Genres')
        print('4. Exit')
        
        # getting choices
        choice = choices(4)
        print('*'* 80)
        
        # if choice is invalid
        if not choice: continue
        
        #if choice is exit 
        elif choice == 4: break
            
        else:
            # search by choice
            search(choice)

def search(choice):
    
    # Print input message as per choice
    tag     = 'Name' if choice == 1 else ( 'Author' if choice == 2 else 'Genres' )
    msg     = f'Enter the {tag}: '
    
    # getting word to be searched
    search  = input(msg)
    
    # Fetching book dataset
    books    = read_file()
    
    # list of searched elements will store in list
    searched = []
    max_len  = 0
    
    # looping for all dictionary values 
    for book in books.values():
        
        # getting data as per choice
        tags = book[tag.lower()]

        # if word found in tags
        if search.lower() in tags.lower():
            
            # retrive name of book 
            name     = book['name']
            
            # checking status of availiablity
            status   = '   AVAIL' if book['quantity'] > 0 else 'NOTAVAIL'
            
            # adding in list of searched element 
            searched.append( (name,status) )
            
            # for print purpose
            max_len  = len(name) if len(name) > max_len else max_len
    
    # list of searched element is empty
    if searched == []:
        print('Book not found')
        print('*'*80)
    
    # list of searched element is not empty
    else:
        
        # print tags for genres/author and its value
        if choice != 1:
            tag = tag.upper()
            print(tag,':',search)
        
        # print list of searched element 
        print('-'*(max_len+14))
        print('NAMES'.ljust(max_len),'  STATUS')
        print('-'*(max_len+14))
        
        for name,status in searched:
            
            print(name.ljust(max_len),status)
        
        print('*'*(max_len+14))
        


In [6]:
def check_status(issued,flag = True):
    
    # intialised status as per flag 
    # if flag True return status in bool
    # if flag False return data of book_ids and expire_dates in tuple 
    
    status = True if flag else ()
    
    # loop for every issued book
    for book , date in issued:
      
        # Difference of expire from today
        delta = dt.now() - dt.strptime(date,'%d-%m-%y')
        
        # if difference of days is +ve --> penalty
        if delta.days > 0:
            
            # for check status
            if flag:
                # print penalty message, set status False & Break the loop
                print(book ,'is expired at ', date, 'pay fine')
                status = False
                break
            
            # return list of books_ids and expire_date 
            else:
                # if first entry
                if status == ():
                    status        = (book, date,1)
                    
                # if not first entry
                else:
                    books , dates, count = status
                    books        += ', ' + book
                    dates        += ', ' + date
                    status        = (books, dates , count +1)
                
            
    
    # return status
    return status
    


def book_issue(book_id,stud_id,books,students):
    
    # getting list of issued of that stud_id
    issued     = students[stud_id ]['issued']
    
    # checking status of penalty
    if check_status(issued):
        
        # genrating expire date for that book
        expire = dt.now() + td( days = 28)
        expire = expire.strftime('%d-%m-%y')

        # initizise flag as True
        flag   = True
        
        # loop for all issued ---> Reissued operation 
        for idx in range(len(issued)):
            
            # book_id found in list then reissued , set flag False & break loop
            
            if issued[idx][0] == book_id:
                flag           = False
                issued[idx][1] = expire
                break
        
        # Flag still true    ---> new book issue operation
        if flag: 
            
            # Added book_id, expire in issued 
            issued.append( (book_id,expire) )
            
            # Updating book quantity of that book_id and saving in database
            
            books[ book_id  ]['quantity'] -=  1
            write_file( books )
        
        # Updating student issued of that stud_id and saving in database
        students[stud_id ]['issued']  = issued
        write_file( students, 'student')
        
        # print sucessfully operation message
        print('Book issued sucessfully')


In [11]:

def book_return(book_id,stud_id,books,students):
    
    # Updating student issued of that stud_id and saving in database
    books[book_id]['quantity'] += 1
    write_file(books)
    
    # getting list of issued of that stud_id
    issued    = students[stud_id]['issued']
    
    # loop for all issued        --> Checking Penalty 
    for idx in range(len(issued)):
        
        # finding  book_id in issued list if found at end break loop
        if issued[idx][0] == book_id:
            
            # difference of expire date from today
            delta = dt.now() - dt.strptime(issued[idx][1],'%d-%m-%y')
            
            # difference is +ve Warning message of penalty to student
            if delta.days > 0:
                print('WARNING :Late submission on this',book_id,'pay fine')
            
            # difference is -ve remove book , expire from issued list 
            else:
                issued.remove(issued[idx])
                
                # update issued list of that stud_id and saving in database
                students[stud_id]['issued']  =  issued
                write_file(students,'student')
                print('Book return sucessfully')
            break
  

In [8]:
def blacklist():
    
    students  = read_file('student')
    
    blacklist = { key : [] for key in [ 'id', 'name', 'graduation_year', 'degree', 'books', 'expired' ] }
    
    for stud_id , student in students.items():
        
        book_expired = check_status(student['issued'],False)
        
        if book_expired != ():
            
            blacklist['id'             ].append(stud_id          )
            
            blacklist['name'           ].append(student['name'           ])
            blacklist['graduation_year'].append(student['graduation year'])
            blacklist['degree'         ].append(student['degree'         ])
            
            blacklist['books'          ].append(book_expired[0           ])
            blacklist['expired'        ].append(book_expired[1           ])
    
    file_name = 'Blacklist Student till ' + dt.now().strftime('%d-%m-%y') + '.csv'
    pd.DataFrame(blacklist).to_csv(file_name,index = False)

In [15]:
def pay_fine():
    
    stud_id = None
    
    students      = read_file('student')
     # While Loop until inputs are valid
    while True:
        
        #
        try:
            stud_id = input('Scan / Type student id: ').upper()

            if not stud_id in students :

                print('Student id not found')
                continue

            break
        except:
            print('Try Again ... ') 
            
    issued       = students[stud_id]['issued']
    book_expired = check_status(issued,flag = False)
    
    if book_expired != ():
        
        print('You have to pay fine for', book_expired[2], 'books')
        print('Total Fine', 500 * book_expired[2])
        
        paid = input("Do you want to pay Type 'paid' for done payment ").lower()
        
        if paid == 'paid':
            
            for book_id , expired in issued:
                
                print(book_id,expired)
                
                if book_id in  book_expired[0]:
                    
                    issued.remove([book_id , expired])
            
            students[stud_id]['issued'] = issued
            
            write_file(students,'student')
            
            print('Payment done and Penalty removed')
            
    else:
        print('Cleared')

In [16]:
if __name__ == '__main__':
    
    while True:
        
        print('Welcome to Library Management')
        print('1. Book Avail')
        print('2. Book Issue / Reissue')
        print('3. Book return ')
        print('4. List of Blacklist Student')
        print('5. Pay fine')
        print('6. Exit')
        
        choice = choices(6)
        
        if not choice   : continue
        
        elif choice == 6: break
        
        elif choice == 1: book_avail()
            
        elif choice < 4 :
            
            books            = read_file (         )
            students         = read_file ('student')
            
            
            book_id ,stud_id = inputs()
            
            
            if   choice == 2 : book_issue (book_id ,stud_id, books, students)
                
            else             : book_return(book_id ,stud_id, books, students)
            
        elif choice == 4 : blacklist()
        else             : pay_fine()
            
        print('*'*80) 

Welcome to Library Management
1. Book Avail
2. Book Issue / Reissue
3. Book return 
4. List of Blacklist Student
5. Pay fine
6. Exit
Enter your choice number: 1
Check Availiability
1. Search by Name
2. Search by Author
3. Search by Genres
4. Exit
Enter your choice number: 1
********************************************************************************
Enter the Name: harry
-------------------------------------------------------------------------------------------------------------------------------------------
NAMES                                                                                                                           STATUS
-------------------------------------------------------------------------------------------------------------------------------------------
Harry Potter and the Philosopher's Stone                                                                                         AVAIL
Samsara: Enter the Valley of the Gods ("India's answer to Harry Potter")

Enter your choice number: 3
********************************************************************************
Enter the Genres: Exam
GENRES : Exam
-------------------------------------------------------------------------------------------------------------------------------------------
NAMES                                                                                                                           STATUS
-------------------------------------------------------------------------------------------------------------------------------------------
KVS PEDAGOGY MASTER BOOK (BILINGUAL) THEORY with PRACTICE MCQ's                                                                  AVAIL
Indian Polity ( English| 6th Revised Edition) | UPSC | Civil Services Exam | State Administrative Exams                          AVAIL
BlackBook of General Awareness January 2023 by Nikhil Gupta                                                                      AVAIL
Examcart KVS PRT Pedagogy English 

Enter your choice number: 4
********************************************************************************
********************************************************************************
Welcome to Library Management
1. Book Avail
2. Book Issue / Reissue
3. Book return 
4. List of Blacklist Student
5. Pay fine
6. Exit
Enter your choice number: 2
Scan / Type Book id: VLB0001
Scan / Type student id: VU1F1819026
Book issued sucessfully
********************************************************************************
Welcome to Library Management
1. Book Avail
2. Book Issue / Reissue
3. Book return 
4. List of Blacklist Student
5. Pay fine
6. Exit
Enter your choice number: 2
Scan / Type Book id: VLB0001
Scan / Type student id: VU1F1819025
VLB2456 is expired at  26-12-22 pay fine
********************************************************************************
Welcome to Library Management
1. Book Avail
2. Book Issue / Reissue
3. Book return 
4. List of Blacklist Student
5. Pay fine
6. Exi